In [177]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

import numpy as np
import nibabel

# Architectures :

## 1) U-Net

In [148]:
# сверткаб можно добавить batch_norm
def double_conv(in_channel, out_channel):
    conv = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=3),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channel, out_channel, kernel_size=3),
        nn.ReLU(inplace=True),
    )
    return conv


# превращает тензор в из одного в нужный размер по центру
def crop_tensor(target_tensor, tensor):
    target_size = target_tensor.size()[2]
    tensor_size = tensor.size()[2]
    delta = tensor_size - target_size
    delta = delta // 2 
    if tensor_size % 2 == 1: # аккуратно! размеры совпадают, но не уверен, что правильно
        return tensor[:, :, delta:tensor_size-delta-1, delta:tensor_size-delta-1] 
    else:
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta] 




class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # левый проход
        self.down_conv1 = double_conv(1, 64)
        self.down_conv2 = double_conv(64, 128)
        self.down_conv3 = double_conv(128, 256)
        self.down_conv4 = double_conv(256, 512)
        self.down_conv5 = double_conv(512, 1024)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        # правый проход
        self.trans1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.up_conv1 = double_conv(1024, 512)
        
        self.trans2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up_conv2 = double_conv(512, 256)
        
        self.trans3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_conv3 = double_conv(256, 128)
        
        self.trans4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_conv4 = double_conv(128, 64)

        # выходной слой
        self.out = nn.Conv2d(64, 2, kernel_size=1)
        
    def crop(self, x, target):
        x = torchvision.transforms.functional.center_crop(x, target)
        return x

    def forward(self, image):

        #forward pass левый
        x1 = self.down_conv1(image)
        x2 = self.maxpool(x1)
        
        x3 = self.down_conv2(x2)
        x4 = self.maxpool(x3)
        
        x5 = self.down_conv3(x4)
        x6 = self.maxpool(x5) 
        
        x7 = self.down_conv4(x6)
        x8 = self.maxpool(x7)
        
        x9 = self.down_conv5(x8)

        # forward pass правый
        x = self.trans1(x9)
        y = crop_tensor(x, x7)
        x = self.up_conv1(torch.cat([x,y], 1))
        print(x.shape)
        
        x = self.trans2(x)
        y = crop_tensor(x, x5)
        x = self.up_conv2(torch.cat([x,y], 1)) # !!! тут были проблемы у crop_image (изза нечетных размеров)
        print(x.shape)
        
        x = self.trans3(x)
        y = crop_tensor(x, x3)
        x = self.up_conv3(torch.cat([x,y], 1))
        print(x.shape)
        
        x = self.trans4(x)
        y = crop_tensor(x, x1)
        x = self.up_conv4(torch.cat([x,y], 1))
        print(x.shape)
        
        x = self.out(x)
                
        return x

# Nii-unpacker

In [175]:
from os import listdir
from os.path import isfile, join

i = 1

mypath = r"C:\Users\Alexey\Desktop\Thesis Paper 2020\MICCAI_BraTS2020_TrainingData\BraTS20_Training_00" + f'{i}'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
img = nibabel.load(mypath + '\\' + onlyfiles[0])
data = img.get_fdata()
data = torch.Tensor(data)
data.size()
data = data.permute(2, 0, 1)
data.shape

torch.Size([155, 240, 240])